In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
import numpy as np


In [2]:
local_path = r"D:\GitHub\Projetos\Mestrado\EnergyContext\pdf\appendixa_0.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    raise FileNotFoundError("Upload a PDF file")


In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)


In [4]:
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name="local-rag"
)


OllamaEmbeddings: 100%|██████████| 3/3 [00:08<00:00,  2.81s/it]


In [5]:
'''
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an electrical engineer. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database, and only the database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide an answer combining the most important points of these five versions.
    Original question: {question}"""
)
'''
'''
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an electrical engineer. Your task is to generate a response based on the vector database.Your
    goal is to help the user overcome some of the limitations of the distance-based 
    similarity search. Provide an answer as close as possible to what is in the provided document.
    Original question: {question}"""
)
'''
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""answer_prompt_template = You are an expert in electrial engineering.
    Using only the information provided in the context, answer the following question:
    Original question: {question}""")


In [6]:
local_model = "mistral"
llm = ChatOllama(model=local_model)
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)


In [7]:
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [8]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [9]:
#chain.invoke("What are technologies used to provide indoor environmental comfort?")

In [10]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_metric
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# --- Data ---

test_queries = [
    "What is the relationship between Megawatt (MW) and Megawatt-hour (MWh)?",
    "Who prepared the Annual Energy Outlook?",
    "What is the Clean Air Act?",
    "What is an Electric Generating Unit?",
    "How Electricity Dispatch Models optimize the dispatch of a system?",
    "Who maintains the Emissions & Generation Resource Integrated Database?",
    "Give me examples of Energy Efficiency",
    "What are ISOs?",
    "What are Power Pools?",
    "What does the Public Utilities Commission or Public Service Commission do?",
    "What does Electricity Dispatch Models simulate?",
]

expected_answers = [
    "A megawatt (MW) is a unit of power representing the rate at which energy is used or generated, while a megawatt-hour (MWh) is a unit of energy representing the total amount of energy used or generated over an hour.",
    "The U.S. Department of Energy’s Energy Information Administration (EIA)",
    "The Clean Air Act (CAA) is the law that defines the U.S. Environmental Protection Agency’s responsibilities for protecting and improving the nation's air quality and the stratospheric ozone layer",
    "An entity that supplies electricity to the electricity system relying on a variety of fuels.",
    "These models optimize the dispatch of the system based on the variable costs of each resource and any operational constraints that have been entered into the model.",
    "The U.S. Environmental Protection Agency",
    "High-efficiency appliances; efficient lighting; high-efficiency heating, ventilating and air conditioning systems or control modifications; efficient building design; advanced electric motor drives; combined heat and power; and heat recovery systems.",
    "Independent System Operators",
    "A power pool is an association of two or more interconnected electric systems that agree to coordinate operations and planning for improved reliability and efficiencies.",
    "Regulates the rates and services of a public utility.",
    "Simulate the dynamic operation of the electric system, generally on a least-cost system dispatch",
]


# --- Evaluation Functions ---

def categorize_score(score, thresholds):
    if score < thresholds['low']:
        return 'low'
    elif score < thresholds['mid']:
        return 'mid'
    else:
        return 'high'

def calculate_scores(prediction, reference):
    bleu_score = sentence_bleu([reference.split()], prediction.split(), smoothing_function=SmoothingFunction().method1)

    rouge_scores = rouge.compute(predictions=[prediction], references=[reference])
    # Extract individual ROUGE scores
    rouge1 = rouge_scores['rouge1'].mid.fmeasure
    rouge2 = rouge_scores['rouge2'].mid.fmeasure
    rougeL = rouge_scores['rougeL'].mid.fmeasure

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode([prediction, reference], convert_to_tensor=True)
    semantic_similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()

    return {
        'bleu': bleu_score,
        'rouge1': rouge1,
        'rouge2': rouge2,
        'rougeL': rougeL,
        'semantic_similarity': semantic_similarity,
    }

def evaluate_rag_system(chain, test_queries, expected_answers):
    results = []

    for query, expected in zip(test_queries, expected_answers):
        prediction = chain.invoke(query) 
        scores = calculate_scores(prediction, expected)
        # Include all relevant data in the results
        results.append({'query': query, 'expected': expected, 'predicted': prediction, 
                        **scores})  # Unpack all scores directly

    return results

# --- Main Evaluation ---

rouge = load_metric("rouge")

# Assuming 'chain' is your RAG system
evaluation_results = evaluate_rag_system(chain, test_queries, expected_answers)

# --- Create DataFrame ---
df = pd.DataFrame(evaluation_results)

# --- Calculate and Print Averages ---

avg_bleu = df['bleu'].mean()
avg_semantic_similarity = df['semantic_similarity'].mean()

# Calculate and print average ROUGE scores
avg_rouge_scores = {}
for rouge_type in ['rouge1', 'rouge2', 'rougeL']:
    avg_score = df[rouge_type].mean()
    avg_rouge_scores[rouge_type] = avg_score
    print(f"Average {rouge_type.upper()}: {avg_score:.2f}")

print(f"\nAverage BLEU Score: {avg_bleu:.2f}")
print(f"Average Semantic Similarity: {avg_semantic_similarity:.2f}\n")

# --- Save Results ---
df.to_excel('rouge_results.xlsx', index=False) 



C:\Users\Matheus\AppData\Local\Temp\ipykernel_18412\2521075999.py:83: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
d:\ProgramData\anaconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.41s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbedd

Average ROUGE1: 0.23
Average ROUGE2: 0.11
Average ROUGEL: 0.18

Average BLEU Score: 0.03
Average Semantic Similarity: 0.65

